<h1> City of Toronto Neighborhood Analysis </h1>

<h1>*********************************Part 1**************************</h1>

In [1]:
#from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
import pandas as pd
import numpy as np

<h2> First task is to webscrap the data from wikipedia</h2>

In [2]:
#wikipedia page with Toronto Neighborhood Data
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' 

In [4]:
#Get the Requested page and convert it to text
data  = requests.get(url).text 

In [5]:
#Using Beautibul Soup to get the list of all dataframes in the requested website
dataframe_list = pd.read_html(url, flavor='bs4')
len(dataframe_list)

3

In [6]:
#checkink the first DataFrame out of 3
dataframe_list[0]

,0,1,2,3,4,5,6,7,8
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park(Ontario Provincial Government),M8ANot assigned,M9AEtobicoke(Islington Avenue)
1,M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),M7BNot assigned,M8BNot assigned,M9BEtobicoke(West Deane Park / Princess Garden...
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M2CNot assigned,M3CNorth York(Don Mills)South(Flemingdon Park),M4CEast York(Woodbine Heights),M5CDowntown Toronto(St. James Town),M6CYork(Humewood-Cedarvale),M7CNot assigned,M8CNot assigned,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,M1EScarborough(Guildwood / Morningside / West ...,M2ENot assigned,M3ENot assigned,M4EEast Toronto(The Beaches),M5EDowntown Toronto(Berczy Park),M6EYork(Caledonia-Fairbanks),M7ENot assigned,M8ENot assigned,M9ENot assigned
4,M1GScarborough(Woburn),M2GNot assigned,M3GNot assigned,M4GEast York(Leaside),M5GDowntown Toronto(Central Bay Street),M6GDowntown Toronto(Christie),M7GNot assigned,M8GNot assigned,M9GNot assigned
5,M1HScarborough(Cedarbrae),M2HNorth York(Hillcrest Village),M3HNorth York(Bathurst Manor / Wilson Heights ...,M4HEast York(Thorncliffe Park),M5HDowntown Toronto(Richmond / Adelaide / King),M6HWest Toronto(Dufferin / Dovercourt Village),M7HNot assigned,M8HNot assigned,M9HNot assigned
6,M1JScarborough(Scarborough Village),M2JNorth York(Fairview / Henry Farm / Oriole),M3JNorth York(Northwood Park / York University),M4JEast YorkEast Toronto(The Danforth East),M5JDowntown Toronto(Harbourfront East / Union ...,M6JWest Toronto(Little Portugal / Trinity),M7JNot assigned,M8JNot assigned,M9JNot assigned
7,M1KScarborough(Kennedy Park / Ionview / East B...,M2KNorth York(Bayview Village),M3KNorth York(Downsview)East (CFB Toronto),M4KEast Toronto(The Danforth West / Riverdale),M5KDowntown Toronto(Toronto Dominion Centre / ...,M6KWest Toronto(Brockton / Parkdale Village / ...,M7KNot assigned,M8KNot assigned,M9KNot assigned
8,M1LScarborough(Golden Mile / Clairlea / Oakridge),M2LNorth York(York Mills / Silver Hills),M3LNorth York(Downsview)West,M4LEast Toronto(India Bazaar / The Beaches West),M5LDowntown Toronto(Commerce Court / Victoria ...,M6LNorth York(North Park / Maple Leaf Park / U...,M7LNot assigned,M8LNot assigned,M9LNorth York(Humber Summit)
9,M1MScarborough(Cliffside / Cliffcrest / Scarbo...,M2MNorth York(Willowdale / Newtonbrook),M3MNorth York(Downsview)Central,M4MEast Toronto(Studio District),M5MNorth York(Bedford Park / Lawrence Manor East),M6MYork(Del Ray / Mount Dennis / Keelsdale and...,M7MNot assigned,M8MNot assigned,M9MNorth York(Humberlea / Emery)


<h4>The first dataframe seems to contain the required data. 
Each element of the dataframe contains a string which has first 3 character as the postal code, next the Borough Name and next inside the bracket are the Neighbourhood names separated with '/' </h4>

In [7]:
dataframe_list[0].shape


(20, 9)

<h3>First we will create a dataFrame with 2 columns one with postal code and other with rest all Informations named RestInfo.</h3>
<h4>Later we will seperate the Borough and Neighborhood names from the RestInfo column</h4>

In [8]:
df = pd.DataFrame(columns=['PostalCode','RestInfo']) #Creating the dataframe with 2 columns
df_temp = dataframe_list[0]

In [9]:
#Iterating through each elements and adding the corresponding PostalCode and RestInfo in our DataFrame df
for rows in range(df_temp.shape[0]):
        for columns in range(df_temp.shape[1]):
            key=df_temp[columns][rows].lstrip()[0:3] #using the lstrip() func to ensure no blank space is counted. Then sing the 1st 3 character as postalcode
            val=df_temp[columns][rows].lstrip()[3:] #assigning rest of the string as RestInfo
            new_row = {'PostalCode':key,'RestInfo':val}
            df = df.append(new_row, ignore_index=True)
df

,PostalCode,RestInfo
0,M1A,Not assigned
1,M2A,Not assigned
2,M3A,North York(Parkwoods)
3,M4A,North York(Victoria Village)
4,M5A,Downtown Toronto(Regent Park / Harbourfront)
...,...,...
175,M5Z,Not assigned
176,M6Z,Not assigned
177,M7Z,Not assigned
178,M8Z,Etobicoke(Mimico NW / The Queensway West / Sou...


In [10]:
# in RestInfo Column anything before the bracket '(' is Borough and Info inside the bracket are neigborhood
# So we split the RestInfo in two part (seperated by "(" ), the first part is Borough and Second id Neighborhoods
# We use two separate functions to get the result
def find_Borough(x):
    if '(' in x:
        return x.split('(')[0]
    else:
        return x
    
    
def find_Neighb(x):
    if '(' in x:
        str_len = len(x.split('(')[1])
        return x.split('(')[1][:str_len-1] # skipping the last character as it would be the closing bracket ')'.
    else:
        return 

In [11]:
#use the functions defines above to seperate Borough and Neighborhood
df['Borough'] = df['RestInfo'].apply(find_Borough)
df['Neighbourhood'] = df['RestInfo'].apply(find_Neighb)
df.head()

,PostalCode,RestInfo,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned,None
1,M2A,Not assigned,Not assigned,None
2,M3A,North York(Parkwoods),North York,Parkwoods
3,M4A,North York(Victoria Village),North York,Victoria Village
4,M5A,Downtown Toronto(Regent Park / Harbourfront),Downtown Toronto,Regent Park / Harbourfront


In [12]:
#dropping the RestInfo column
df.drop(columns=['RestInfo'],axis=1, inplace=True)
df.tail()

,PostalCode,Borough,Neighbourhood
175,M5Z,Not assigned,None
176,M6Z,Not assigned,None
177,M7Z,Not assigned,None
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...
179,M9Z,Not assigned,None


In [13]:
#replace the slash '/' with ',' in Neighboorhood columns
nl=None
df['Neighbourhood'] = df['Neighbourhood'].apply(lambda x: None if x==nl else x.replace('/',','))
df['Neighbourhood']

0                                                   None
1                                                   None
2                                              Parkwoods
3                                       Victoria Village
4                             Regent Park , Harbourfront
                             ...                        
175                                                 None
176                                                 None
177                                                 None
178    Mimico NW , The Queensway West , South of Bloo...
179                                                 None
Name: Neighbourhood, Length: 180, dtype: object

<h3>As per the Problem Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.</h3>
<H2>Now we need to delete the rows where Borough is Not assigned.</H2>

In [14]:
df.drop(df[df['Borough']=='Not assigned'].index,inplace =True)
df.reset_index(drop=True, inplace =True)
df

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."


<h2> Check if any Postal Code is listed twice </h2>

In [15]:
len(df['PostalCode'].unique())

103

<h4>Since there are 103 unique value of Postal Code and there are 103 rows in the table, there seems no double entry of postal code</h4>

<h3>According to the problem If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
So we check: </h3>
<h2> If there is a Not Assigned Neighborhood</h2>

In [16]:
df[df['Neighbourhood'].isnull()]

,PostalCode,Borough,Neighbourhood


In [17]:
df[df['Neighbourhood']==" "]

,PostalCode,Borough,Neighbourhood


In [18]:
df[df['Neighbourhood']== np.nan]

,PostalCode,Borough,Neighbourhood


<h4> so according to the above tests all Neighbourhood is assigned to some value and there is no Neighborhood with null value or Not assigned value. Therefor no further operation need to be done here </h4>

In [19]:
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [20]:
df.shape

(103, 3)

In [21]:
df.to_csv('TorontoPostalCode.csv',index=False) #saving the file so that the whole part1 need not to be run again for part 2

<h1>***************************Part 2 *********************************</h1>

In [1]:
#import geocoder # import geocoder
!conda install -c conda-forge geopy --yes


Solving environment: ...working... done

# All requested packages already installed.



In [2]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: ...working... done

# All requested packages already installed.



In [3]:
from geopy.geocoders import Nominatim
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


In [4]:
df1=pd.read_csv('TorontoPostalCode.csv')
df1.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [5]:
#Function to get latitude and Longitude using geolocator
def get_coordinates(postal_code):
    geolocator = Nominatim(user_agent="foursquare_agent")
    g = geolocator.geocode('{}, Toronto, Ontario'.format(postal_code))
    try:
        latitude = g.latitude
        longitude = g.longitude
    except:
        return None
    return latitude,longitude

In [7]:
#checking if the function works correctly
get_coordinates('M7A')

(43.6534817, -79.3839347)

In [8]:
df1['Latitude_Longitude']= df1['PostalCode'].apply(get_coordinates)

In [9]:
df1['Latitude_Longitude']

0                    (43.6534817, -79.3839347)
1                                         None
2                                         None
3                                         None
4                    (43.6534817, -79.3839347)
5                    (43.6534817, -79.3839347)
6                    (43.6534817, -79.3839347)
7                                         None
8                                         None
9                                         None
10                                        None
11                                        None
12                   (43.6534817, -79.3839347)
13                                        None
14                                        None
15                                        None
16                                        None
17     (43.64410993066079, -79.58890692194828)
18                                        None
19                                        None
20                   (43.6421064, -79.3774455)
21           

In [10]:
len(df1['Latitude_Longitude'].unique())

18

coordinates of only 18 postalArea could be found 

In [11]:
df1.drop(columns=['Latitude_Longitude'], inplace = True)

<h4> Due to time out error and inefficiency of the above Geolocator service we will skip using geolocator and use the Geospatial_Coordinates.csv as provided to find the latitude and longitude</h4>

In [12]:
lat_lng_file = pd.read_csv('Geospatial_Coordinates.csv')
df2=pd.merge(df1, lat_lng_file, how='inner', left_on='PostalCode', right_on='Postal Code',left_index=False, right_index=False)
df2.head()

,PostalCode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,M7A,43.662301,-79.389494


In [13]:
df2.drop(columns=['Postal Code'],inplace=True)
df2.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [14]:
df2.shape

(103, 5)

<h1>******************************Part 3*********************************</h1>

In [15]:
import requests 

<h4> Finding all the Borough with name 'Toronto' in it</h4>

In [16]:

df3 = df2[df2['Borough'].apply(lambda x: True if 'Toronto' in x else False)]
df3.reset_index(drop=True)
df3

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond , Adelaide , King",43.650571,-79.384568
31,M6H,West Toronto,"Dufferin , Dovercourt Village",43.669005,-79.442259
35,M4J,East YorkEast Toronto,The Danforth East,43.685347,-79.338106


In [86]:
len(df3['Borough'].unique())

7

<h4> Now we prepare the Credentials to access Foursquare API</h4>

In [18]:
CLIENT_ID = 'EDVNUDPMPCTF4YZYEHJ5HF0UQEX5LLKI2RZ5NT21ELCZ5DON' # your Foursquare ID
CLIENT_SECRET = 'HW5DJH3G5BSDXF1JAMDLI5SIQZZIFDIMTPOBQNOACQ3CWHLT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EDVNUDPMPCTF4YZYEHJ5HF0UQEX5LLKI2RZ5NT21ELCZ5DON
CLIENT_SECRET:HW5DJH3G5BSDXF1JAMDLI5SIQZZIFDIMTPOBQNOACQ3CWHLT


<h4>Now, let's get the top 100 venues that are in area with Postal Code 'M5A' within a radius of 500 meters.</h4>

In [19]:
neighborhood_latitude=df3[(df3['PostalCode']=='M5A').values]['Latitude'].values[0]
neighborhood_longitude=df3[(df3['PostalCode']=='M5A').values]['Longitude'].values[0]

In [20]:
neighborhood_latitude,neighborhood_longitude

(43.6542599, -79.3606359)

Preparing the url

In [21]:

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=EDVNUDPMPCTF4YZYEHJ5HF0UQEX5LLKI2RZ5NT21ELCZ5DON&client_secret=HW5DJH3G5BSDXF1JAMDLI5SIQZZIFDIMTPOBQNOACQ3CWHLT&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

Requesting the Resultant JSON file

In [22]:
results = requests.get(url).json()
json_formatted_str = json.dumps(results, indent=6)
print(json_formatted_str)

{
      "meta": {
            "code": 200,
            "requestId": "60deca4c28ef2f25c43e57fe"
      },
      "response": {
            "headerLocation": "Corktown",
            "headerFullLocation": "Corktown, Toronto",
            "headerLocationGranularity": "neighborhood",
            "totalResults": 42,
            "suggestedBounds": {
                  "ne": {
                        "lat": 43.6587599045,
                        "lng": -79.3544279001486
                  },
                  "sw": {
                        "lat": 43.6497598955,
                        "lng": -79.36684389985142
                  }
            },
            "groups": [
                  {
                        "type": "Recommended Places",
                        "name": "recommended",
                        "items": [
                              {
                                    "reasons": {
                                          "count": 0,
                                          "

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Converting the data in the JSON format to DataFrame 

In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-24-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Tandem Coffee,Coffee Shop,43.653559,-79.361809
1,Roselle Desserts,Bakery,43.653447,-79.362017
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


<h4> Now we will repeat the same process (as done above for only one postal code 'M5A') for all postal Code </h4>

In [25]:
#definning a function to extract data from JSON to DataFrame for all Postal Code
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'Postal Latitude', 
                  'Postal Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
toronto_venues = getNearbyVenues(names=df3['PostalCode'],
                                   latitudes=df3['Latitude'],
                                   longitudes=df3['Longitude']
                                  )

M5A
M5B
M5C
M4E
M5E
M5G
M6G
M5H
M6H
M4J
M5J
M6J
M4K
M5K
M6K
M4L
M5L
M4M
M4N
M5N
M4P
M5P
M6P
M4R
M5R
M6R
M4S
M5S
M6S
M4T
M5T
M4V
M5V
M4W
M5W
M4X
M5X
M4Y
M7Y


In [27]:
# Checking the size of the DataFrame
print(toronto_venues.shape)
toronto_venues.head(15)

(1587, 7)


,PostalCode,Postal Latitude,Postal Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5A,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,M5A,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,M5A,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,M5A,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,M5A,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
5,M5A,43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site
6,M5A,43.65426,-79.360636,Distillery Sunday Market,43.650075,-79.361832,Farmers Market
7,M5A,43.65426,-79.360636,SOMA chocolatemaker,43.650622,-79.358127,Chocolate Shop
8,M5A,43.65426,-79.360636,Arvo,43.649963,-79.361442,Coffee Shop
9,M5A,43.65426,-79.360636,Sumach Espresso,43.658135,-79.359515,Coffee Shop


Check how many results are obtained for each Postal Code

In [28]:
toronto_venues.groupby('PostalCode').count()

,Postal Latitude,Postal Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode,,,,,,
M4E,4,4,4,4,4,4
M4J,2,2,2,2,2,2
M4K,42,42,42,42,42,42
M4L,23,23,23,23,23,23
M4M,36,36,36,36,36,36
M4N,3,3,3,3,3,3
M4P,7,7,7,7,7,7
M4R,20,20,20,20,20,20
M4S,37,37,37,37,37,37


In [30]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 231 uniques categories.


Use one Hot Encoding on all categories for ease of applying ML algo like KNearest Neighbor

In [31]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,PostalCode,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,M5A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M5A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [32]:
toronto_onehot.shape

(1587, 232)

#### Next, let's group rows by Postal Codes and by taking the mean of the frequency of occurrence of each category

In [33]:
toronto_grouped = toronto_onehot.groupby('PostalCode').mean().reset_index()
toronto_grouped

,PostalCode,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,M4E,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

#### Let's print each Postal Area along with the top 5 most common venues


In [35]:
num_top_venues = 5

for hood in toronto_grouped['PostalCode']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['PostalCode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4E----
               venue  freq
0       Neighborhood  0.25
1  Health Food Store  0.25
2                Pub  0.25
3              Trail  0.25
4     Adult Boutique  0.00


----M4J----
                 venue  freq
0                 Park   0.5
1    Convenience Store   0.5
2  Monument / Landmark   0.0
3               Lounge   0.0
4     Malay Restaurant   0.0


----M4K----
                    venue  freq
0        Greek Restaurant  0.17
1             Coffee Shop  0.10
2      Italian Restaurant  0.07
3  Furniture / Home Store  0.05
4              Restaurant  0.05


----M4L----
                  venue  freq
0                  Park  0.13
1  Fast Food Restaurant  0.09
2     Food & Drink Shop  0.04
3                   Pub  0.04
4               Brewery  0.04


----M4M----
                 venue  freq
0          Coffee Shop  0.08
1              Brewery  0.06
2               Bakery  0.06
3            Gastropub  0.06
4  American Restaurant  0.06


----M4N----
            venue  freq
0           

#### Let's put that into a _pandas_ dataframe

First, let's write a function to sort the venues in descending order.


In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 5 venues for each neighborhood.

In [38]:
import numpy as np

In [77]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,Neighborhood,Pub,Health Food Store,Trail,Donut Shop
1,M4J,Park,Convenience Store,Diner,Farmers Market,Falafel Restaurant
2,M4K,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Ice Cream Shop
3,M4L,Park,Fast Food Restaurant,Brewery,Italian Restaurant,Fish & Chips Shop
4,M4M,Coffee Shop,Bakery,Café,American Restaurant,Gastropub


Run _k_-means to cluster the neighborhood into 5 clusters.

In [78]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 0, 0, 0, 2, 0, 0, 0, 1])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [79]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df3

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('PostalCode'), on='PostalCode')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Park,Bakery,Breakfast Spot
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Hotel,Sandwich Place
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Beer Bar,Cocktail Bar,Gastropub
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Neighborhood,Pub,Health Food Store,Trail,Donut Shop
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Beer Bar


In [80]:
latitude,longitude = get_coordinates("Toronto City")

In [81]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h4>Cluster 1</h4>

In [88]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0,1]+ list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,M5A,Downtown Toronto,0,Coffee Shop,Pub,Park,Bakery,Breakfast Spot
9,M5B,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Hotel,Sandwich Place
15,M5C,Downtown Toronto,0,Coffee Shop,Café,Beer Bar,Cocktail Bar,Gastropub
19,M4E,East Toronto,0,Neighborhood,Pub,Health Food Store,Trail,Donut Shop
20,M5E,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Beer Bar
24,M5G,Downtown Toronto,0,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Salad Place
25,M6G,Downtown Toronto,0,Grocery Store,Café,Park,Italian Restaurant,Baby Store
30,M5H,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Thai Restaurant,Deli / Bodega
31,M6H,West Toronto,0,Pharmacy,Bakery,Grocery Store,Brewery,Café
36,M5J,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Scenic Lookout,Hotel


<h4>Cluster 2</h4>

In [89]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0,1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
83,M4T,Central Toronto,1,Lawyer,Playground,Diner,Farmers Market,Falafel Restaurant


<h4>Cluster 3</h4>

In [90]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0,1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
35,M4J,East YorkEast Toronto,2,Park,Convenience Store,Diner,Farmers Market,Falafel Restaurant
61,M4N,Central Toronto,2,Park,Bus Line,Swim School,Comic Shop,Discount Store
68,M5P,Central Toronto,2,Park,Sushi Restaurant,Jewelry Store,Trail,Falafel Restaurant
91,M4W,Downtown Toronto,2,Park,Trail,Playground,Dessert Shop,Falafel Restaurant


We get 3 clusters with 2nd cluster having only one postal code of 'M4T' which seems different from all Postal areas as it do not have any Park or Restaurant or Cofee shops in its top 5 Venues. Thus it seems more formal or Non Residential area